In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/


cp: cannot stat 'kaggle.json': No such file or directory


In [2]:
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
100% 331M/331M [00:17<00:00, 22.2MB/s]
100% 331M/331M [00:17<00:00, 19.3MB/s]


In [3]:
import zipfile
zip = zipfile.ZipFile('utkface-new.zip', 'r')
zip.extractall("/content")
zip.close()

In [4]:
import os
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator


In [5]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [6]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [7]:
len(age)

23708

In [8]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [9]:
df

,age,gender,img
0,26,0,26_0_1_20170116192332618.jpg.chip.jpg
1,36,0,36_0_2_20170104202606050.jpg.chip.jpg
2,37,1,37_1_0_20170117140113541.jpg.chip.jpg
3,21,1,21_1_3_20170104231324474.jpg.chip.jpg
4,36,1,36_1_0_20170109141845861.jpg.chip.jpg
...,...,...,...
23703,72,1,72_1_0_20170110122624490.jpg.chip.jpg
23704,41,1,41_1_1_20170117021604893.jpg.chip.jpg
23705,42,0,42_0_0_20170104204628725.jpg.chip.jpg
23706,3,1,3_1_0_20170109193421750.jpg.chip.jpg


In [10]:
df.shape

(23708, 3)

In [11]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [12]:
train_df.shape

(20000, 3)

In [13]:
test_df.shape

(3708, 3)

In [14]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [15]:
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                  class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [16]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [17]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

94765736/94765736 [==============================] - 5s 0us/step


In [18]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [19]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [20]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [21]:
model.fit(train_generator, batch_size=32, epochs=10, validation_data=test_generator)

Epoch 1/10
625/625 [==============================] - 264s 408ms/step - loss: 98.0239 - age_loss: 15.4608 - gender_loss: 0.8340 - age_mae: 15.4608 - gender_accuracy: 0.5074 - val_loss: 83.3186 - val_age_loss: 14.8567 - val_gender_loss: 0.6915 - val_age_mae: 14.8567 - val_gender_accuracy: 0.5302
Epoch 2/10
625/625 [==============================] - 256s 409ms/step - loss: 84.0992 - age_loss: 15.0443 - gender_loss: 0.6975 - age_mae: 15.0443 - gender_accuracy: 0.5210 - val_loss: 83.4970 - val_age_loss: 15.0292 - val_gender_loss: 0.6916 - val_age_mae: 15.0292 - val_gender_accuracy: 0.5302
Epoch 3/10
625/625 [==============================] - 253s 405ms/step - loss: 83.4306 - age_loss: 14.8930 - gender_loss: 0.6923 - age_mae: 14.8930 - gender_accuracy: 0.5211 - val_loss: 83.0060 - val_age_loss: 14.5320 - val_gender_loss: 0.6917 - val_age_mae: 14.5320 - val_gender_accuracy: 0.5291
Epoch 4/10
625/625 [==============================] - 254s 406ms/step - loss: 83.4527 - age_loss: 14.8402 - gend